#### 提升CPU利用率1
- 在计算$a+b$之前，需要准备数据
  - 主内存->L3->L3->L1->寄存器
    - L1访问延时：0.5ns
    - L2访问延时：7ns（$4\times L1$）
    - 主内存访问延迟：100ns（$200\times L1$）
- **提升空间和时间的内存本地性**
  - 时间：重用数据使得保持它们在缓存里
  - 空间：按序读写数据使得可以预读取
- *样例分析*
  - 如果一个矩阵时按行存储（行上是连续地址），访问一行会比访问一列要快
    - CPU一次读取64字节（缓存线），float32是4个字节
    - CPU会‘聪明的’提前读取下一个（缓存线）

#### 提升CPU利用率2
- 高端CPU有几十个核
- **并行来利用所有核**
  - 超线程不一定提升性能，因为共享寄存器
- 样例分析
  ```
  for i in range(len(a));
    c[i] = a[i] + b[i]
  ```
  `c = a + b`
  - 上面要比下面慢（python）
    - 上面调用n次函数，每次调用都有开销
    - 下面很容易被并行

#### CPU vs GPU
|  核         | 6 / 64           | 2K / 4K          |
|--------------|-------------------|-------------------|
| TFLOPS       | 0.2 / 1           | 10 / 100         |
| 内存大小     | 32 GB / 1 TB      | 16 GB / 32 GB    |
| 内存带宽     | 30 GB/s / 100 GB/s | 400 GB/s / 1 TB/s |
| 控制流       | 强               | 弱               |

- 内存带宽：计算机系统中**内存与处理器之间数据传输的速率**，通常以每秒传输的字节数为单位

#### 提升GPU利用率
- 并行
  - 使用数千个线程
- 内存本地性
  - 缓存更小，架构更加简单（内存带宽会高）
- 少用控制语句
  - 支持有限
  - 同步开销很大

#### CPU/GPU带宽
- 不要频繁在CPU和GPU之间传输数据：带宽限制，还有同步开销
  - GPU和CPU之间的带宽小，尤其是多GPU更慢

#### CPU/GPU高性能计算编程
- CPU：C++或者任何高性能语言
  - 编译器成熟
- GPU
  - Nvidia上用CUDA
    - 编译器和驱动成熟
  - 其他用OpenCL
    - 质量取决于硬件厂商

#### 总结
- CPU：可以处理通用计算。性能优化考虑数据读写效率和多线程
- GPU：使用更多的小核和更好的内存带宽，适合能大规模并行的计算任务

- 计算机的关键部件组成：
  - 一个处理器（CPU）
  - 内存（随机访问存储，RAM）
  - 一个或多个以太网连接
  - 高速扩展总线（PCle），用于系统连接一个多个GPU
  - 持久性存储设备，如磁盘驱动器、固态驱动器，多数情况用高速扩张总线连接

在计算机上运行代码时，需将数据转移到处理器（CPU或GPU）上执行计算，然后再将结果从处理器移回到随机访问存储和持久存储器中
- 为了获取良好的性能，要确保每一步工作能无缝连接，而不要系统中的一部分成为主要的瓶颈
  - 如不能加速加载图像，处理器会无事可做
  - 不能快速移动矩阵到CPU（GPU）上，则CPU（GPU）就会无法全速运行
  - 如果希望子啊网络上同步多台计算机，则网络就不应该拖累计算速度

- 内存
  - CPU内存
    - 向内存发送地址并设置传输大约要100ns，每个后续传输只需要0.2ns——使用突发模式读取和写入，尽量避免随机访问
    - 多个物理存储体允许更高的并行度，提升随机读取效率
  - GPU内存
    - GPU需要更高的内存带宽，因为处理单元更多，通常采用更宽的总线和更高性能的内存
    - GPU内存成本高，容量较小，但速度更快，适合高吞吐量计算

- 硬盘驱动器（HDD）
  - 具备*容量大、成本低*的优势，但存在*读取延迟高和故障率较高*的缺点
  - 逐渐转为归档和低级存储，不适合高性能需求的数据访问

- 固态驱动器（SSD）
  - 使用闪存存储数据，性能更高
  - 但写入效率低，写入速度慢于读取，磨损问题

- CPU（central processing unit）
  - 是任何计算机的核心
  - 处理器核心、总线、缓存（相比于主内存实现更高的读取带宽和更低的延迟内存访问）、向量处理单元
  - 缓存
    - 缓存层次结构：寄存器，一级缓存（L1），二级缓存（L2），三级缓存（L3）
    - 作用：弥补主内存带宽的不足，将数据预先加载到靠近 CPU 的位置，以提高数据访问效率，减少处理器的等待时间
    - 优化：需要合理预判数据访问模式（如顺序遍历、局部访问）来提升性能
    - 限制：增加芯片面积、缓存未命中的代价较高，尤其在多处理器环境下可能导致效率降低，如“错误共享”问题

- GPU和其他加速卡
  - GPU对深度学习的重要性：GPU 的高并行计算能力推动了深度学习的发展，制造商也因深度学习而受益。GPU 是深度学习中最适合的硬件，加速了复杂的计算。
  - 训练和推断的优化差异
    - 推断只需向前传播，通常使用较低精度，不需要存储中间结果，因此优化侧重于效率
    - 训练则需要保存中间结果以计算梯度，要求较高的精度和内存带宽
  - GPU设计策略
    - 通过向量和矩阵运算提升性能，GPU中加入了张量核心，专门优化小型矩阵运算
    - Turing架构包含多个流式多处理器（SM），每个处理块有整数单元、浮点单元和张量核，适合深度学习需求
  - 可扩展性与模块化设计：GPU 设计允许独立添加或删除模块，方便调整芯片规模和处理不合格的模块，这也提升了芯片良品率
  - GPU的局限性：对稀疏数据和中断处理效果较差

- 网络和总线：用于数据同步处理，尤其是设备不足以优化的时候
  - PCle：高带宽，低延迟，用于点对点连接，适合大批量数据传输，但GPU连接数量有限
  - 以太网：成本低、弹性强、覆盖范围广，带宽较低，适合长距离连接，但有数据传输开销
  - 交换机：连接多个设备并实现全带宽点对点连接，适用于大规模集群
  - NVLink：高带宽替代品，专为GPU之间的数据传输设计，速度可达300Gbit/s


### 总结
- 设备有运行开销，数据传输要争取量大而次少。适用于RAM、固态驱动器、网络和GPU
- **矢量化**是性能的关键
- 在训练过程中数据类型过小导致的数值溢出可能是一个问题

# 更多的芯片

- DSP：数字信号处理
  - 为数字信号处理算法设计：点积、卷积、FFT
  - 低功耗、高性能
    - 比移动GPU快5倍，功耗更低
  - VLIW：very long instruction word
    - 一条指令计算上百次乘累加
  - 编程和调试困难
  - 编译器质量良莠不齐

- 可编程陈列（FPGA）
  - 有大量可以编程逻辑单元和可配置的连接
  - 可以配置成计算复杂函数
    - 编程语言：VHDL，Verilog
  - 通常比通用硬件更高效
  - 工具链质量良莠不齐
  - 一次‘编译’需要数小时

-

- 模型复杂度和模型需要的计算量并不成正比
- 能用tensor计算就tensor，就算循环也最好用C
